In [1]:
import pandas as pd
import numpy as np
import warnings

import seaborn as sns
import matplotlib.pyplot as plt

#Plotly Libraris
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots
import plotly.io as pio



warnings.filterwarnings('ignore')

pio.templates.default = "ggplot2"

#dataframe display settings
pd.set_option('display.max_columns', 5000000)
pd.set_option('display.max_rows', 50000000)

c:\Users\pc\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\pc\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
## Data from dataset 3
industry_trends = pd.read_csv('data/dataset3.csv')
industry_trends.rename(columns={"Unnamed: 0": "StartupID", "Unnamed: 6": "City_ZIP"}, inplace=True)
## Data from dataset 4
startup_info = pd.read_csv('data/dataset4.csv')


In [3]:
startup_info['founded_year'] = pd.to_datetime(startup_info['founded_at'], errors = 'coerce').dt.year
startup_info['funding_total_usd'].replace('-', 0, inplace=True)
startup_info['funding_total_usd'] = startup_info['funding_total_usd'].astype(float)
startup_info = startup_info.drop_duplicates()

# Cleanning dataset
clean_startup_info = startup_info.drop(['permalink', 'homepage_url'], axis=1)

# Remove the NaN name
clean_startup_info.dropna(how='any', subset=['name'], axis=0, inplace=True)

#Extracting year value from "first_funding_at" and changing to int
clean_startup_info['first_funding_at'] = clean_startup_info.first_funding_at.str.split("-").str[0]
clean_startup_info['first_funding_at'] = clean_startup_info['first_funding_at'].astype(float)
#Extracting year value from "last_funding_at" and changing to int
clean_startup_info['last_funding_at'] = clean_startup_info.last_funding_at.str.split("-").str[0]
clean_startup_info['last_funding_at'] = clean_startup_info['last_funding_at'].astype(float)

# Treat Nan category
clean_startup_info['category_list'] = clean_startup_info['category_list'].replace(np.nan, 'other')
clean_startup_info.head()

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,founded_year
0,#fame,Media,10000000.0,operating,IND,16,Mumbai,Mumbai,1,NaN,2015.0,2015.0,NaN
1,:Qounter,Application Platforms|Real Time|Social Network...,700000.0,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014.0,2014.0,2014.0
2,"(THE) ONE of THEM,Inc.",Apps|Games|Mobile,3406878.0,operating,NaN,NaN,NaN,NaN,1,NaN,2014.0,2014.0,NaN
3,0-6.com,Curated Web,2000000.0,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008.0,2008.0,2007.0
4,004 Technologies,Software,0.0,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014.0,2014.0,2010.0


In [4]:
corr_startups = clean_startup_info[clean_startup_info['status'] != 'ipo']
corr_startups.reset_index(drop=True, inplace=True)
corr_startups['is_successful'] = corr_startups['status'].apply(lambda x: 1 if x in ['operating', 'acquired'] else 0)

corr_startups.columns
#successful_startups = corr_startups[corr_startups['is_successful']==1]
#successful_startups.apply(lambda x: pd.factorize(x)[0]).corrwith(successful_startups['is_successful'])

Index(['name', 'category_list', 'funding_total_usd', 'status', 'country_code',
       'state_code', 'region', 'city', 'funding_rounds', 'founded_at',
       'first_funding_at', 'last_funding_at', 'founded_year', 'is_successful'],
      dtype='object')

In [ ]:


successful_startups = corr_startups[corr_startups['is_successful']==1]
fail_startups = corr_startups[corr_startups['is_successful']==0]

# Calculate the point-biserial correlation coefficient between each categorical variable and 'is_successful' for both cases
successful_startups = successful_startups.apply(lambda x: pd.factorize(x)[0]).corrwith(successful_startups['is_successful'])
fail_startups = fail_startups.apply(lambda x: pd.factorize(x)[0]).corrwith(fail_startups['is_successful'])


# Plot the correlation matrices
fig, ax = plt.subplots(1, 2, figsize=(20, 8))
sns.heatmap(successful_startups.to_frame(), annot=True, cmap="coolwarm", ax=ax[0], cbar=False, square=True)
ax[0].set_title("Correlation Matrix (is_successful = 1)")
sns.heatmap(fail_startups.to_frame(), annot=True, cmap="coolwarm", ax=ax[1], cbar=False, square=True)
ax[1].set_title("Correlation Matrix (is_successful = 0)")
plt.tight_layout()
plt.show()
